In [115]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
from SALib.analyze import sobol
import warnings
from ema_workbench.analysis import feature_scoring
from ema_workbench.em_framework.salib_samplers import get_SALib_problem
from ema_workbench import (
    Model,
    Policy,
    ema_logging,
    SequentialEvaluator,
    MultiprocessingEvaluator,
    perform_experiments,
    Samplers,
    SequentialEvaluator,
)
from dike_model_function import DikeNetwork
from problem_formulation import get_model_for_problem_formulation, sum_over, sum_over_time
from ema_workbench.analysis import prim
from ema_workbench.em_framework.parameters import Constant
import pickle
warnings.filterwarnings("ignore")

In [139]:
dike_model, planning_steps = get_model_for_problem_formulation(3)

In [119]:
#load it
with open("data\ScenariosOpenExplo", 'rb') as file2:
    Scenarios = pickle.load(file2)

In [122]:
from ema_workbench.em_framework.optimization import ArchiveLogger, EpsilonProgress

# we need to store our results for each seed
results = []
convergences = []

with SequentialEvaluator(dike_model) as evaluator:
    # we run again for 5 seeds
    for i in range(100):
        # we create 2 covergence tracker metrics
        # the archive logger writes the archive to disk for every x nfe
        # the epsilon progress tracks during runtime
        convergence_metrics = [
            ArchiveLogger(
                r"C:\Users\roelo\Videos\archive",
                [l.name for l in dike_model.levers],
                [o.name for o in dike_model.outcomes],
                base_filename=f"{i}.tar.gz",
            ),
            
            EpsilonProgress(),
        ]

        result, convergence = evaluator.optimize(
            nfe=1,
            searchover="levers",
            epsilons=[0.05] * len(dike_model.outcomes),
            convergence=convergence_metrics,
            reference = Scenarios[0]
        )

        results.append(result)
        convergences.append(convergence)

  0%|                                                    | 0/1 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [123]:
from ema_workbench import MultiprocessingEvaluator, ema_logging
from ema_workbench.em_framework.evaluators import BaseEvaluator

ema_logging.log_to_stderr(ema_logging.INFO)
with SequentialEvaluator(dike_model) as evaluator:
    results1 = evaluator.optimize(nfe=100, searchover='levers',
                                  epsilons=[0.001, ] * len(dike_model.outcomes),
                                  reference=Scenarios[0])


100%|████████████████████████████████████████| 100/100 [02:10<00:00,  1.30s/it]
[MainProcess/INFO] optimization completed, found 2 solutions


In [132]:
results1

,0_RfR 0,0_RfR 1,0_RfR 2,1_RfR 0,1_RfR 1,1_RfR 2,2_RfR 0,2_RfR 1,2_RfR 2,3_RfR 0,...,A.4_DikeIncrease 0,A.4_DikeIncrease 1,A.4_DikeIncrease 2,A.5_DikeIncrease 0,A.5_DikeIncrease 1,A.5_DikeIncrease 2,A.5_Expected Number of Deaths,A.5_Expected Annual Damage,RfR Total Costs,Expected Evacuation Costs
0,1,1,0,1,0,0,0,0,0,1,...,9,4,4,3,9,5,0.0,0.0,508200000.0,0.000000
1,0,1,0,0,0,0,1,1,0,0,...,9,7,10,6,4,1,0.0,0.0,267200000.0,59.954408


In [131]:
from optimization_tuned import epsilon_nondominated,HypervolumeMetric, Hypervolume

In [128]:
(list(dike_model.outcomes))

[ScalarOutcome('A.5_Expected Number of Deaths', variable_name=('A.5_Expected Number of Deaths',), function=<function sum_over at 0x000001D50C575760>),
 ScalarOutcome('A.5_Expected Annual Damage', variable_name=('A.5_Expected Annual Damage',), function=<function sum_over at 0x000001D50C575760>),
 ScalarOutcome('RfR Total Costs', variable_name=('RfR Total Costs',), function=<function sum_over at 0x000001D50C575760>),
 ScalarOutcome('Expected Evacuation Costs', variable_name=('Expected Evacuation Costs',), function=<function sum_over at 0x000001D50C575760>)]

In [148]:
    with SequentialEvaluator(dike_model) as evaluator:
    results = evaluator.perform_experiments(scenarios=10,policies=10,uncertainty_sampling=Samplers.LHS)

experiments, outcomes = results
results = pd.DataFrame(results)




  0%|                                                  | 0/100 [00:00<?, ?it/s]


  1%|▍                                         | 1/100 [00:00<01:22,  1.19it/s]


  2%|▊                                         | 2/100 [00:01<01:13,  1.34it/s]


  3%|█▎                                        | 3/100 [00:02<01:03,  1.52it/s]


  4%|█▋                                        | 4/100 [00:02<01:02,  1.53it/s]


  5%|██                                        | 5/100 [00:03<01:00,  1.57it/s]


  6%|██▌                                       | 6/100 [00:03<00:58,  1.62it/s]


  7%|██▉                                       | 7/100 [00:04<00:57,  1.63it/s]


  8%|███▎                                      | 8/100 [00:05<00:55,  1.65it/s]


  9%|███▊                                      | 9/100 [00:05<00:53,  1.71it/s]


 10%|████                                     | 10/100 [00:06<01:11,  1.25it/s]


 11%|████▌                                    | 11/100 [00:08<01:32,  1.04s/it]


 12%|████▉   

AttributeError: 'dict' object has no attribute 'dtype'

In [157]:
experiments

,A.0_ID flood wave shape,A.1_Bmax,A.1_Brate,A.1_pfail,A.2_Bmax,A.2_Brate,A.2_pfail,A.3_Bmax,A.3_Brate,A.3_pfail,...,A.4_DikeIncrease 0,A.4_DikeIncrease 1,A.4_DikeIncrease 2,A.5_DikeIncrease 0,A.5_DikeIncrease 1,A.5_DikeIncrease 2,EWS_DaysToThreat,scenario,policy,model
0,28,334.550720,1.5,0.536400,98.638584,1.5,0.758910,280.435623,10.0,0.493473,...,7,1,4,3,2,3,4,9870,9860,dikesnet
1,2,112.718847,1.5,0.374176,152.862635,1.0,0.593557,239.972876,1.0,0.047630,...,7,1,4,3,2,3,4,9871,9860,dikesnet
2,91,160.342849,1.5,0.137473,278.934201,1.0,0.269154,117.110651,10.0,0.159196,...,7,1,4,3,2,3,4,9872,9860,dikesnet
3,66,253.616129,1.5,0.909950,245.242837,10.0,0.025214,319.344120,1.5,0.705097,...,7,1,4,3,2,3,4,9873,9860,dikesnet
4,119,64.528577,1.0,0.051779,318.661972,1.0,0.419978,312.650051,1.0,0.301750,...,7,1,4,3,2,3,4,9874,9860,dikesnet
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,104,286.858019,10.0,0.647907,90.083315,1.5,0.364053,183.940189,10.0,0.907323,...,4,0,1,8,4,4,1,9875,9869,dikesnet
96,13,268.172666,1.0,0.828675,303.723326,10.0,0.864185,191.973052,1.0,0.237527,...,4,0,1,8,4,4,1,9876,9869,dikesnet
97,121,206.310546,10.0,0.427856,167.921975,10.0,0.917083,73.175780,10.0,0.656236,...,4,0,1,8,4,4,1,9877,9869,dikesnet
98,50,146.798661,10.0,0.246389,219.173105,1.5,0.198945,61.260609,1.5,0.587716,...,4,0,1,8,4,4,1,9878,9869,dikesnet


In [155]:
[type(x) for x in results]

[pandas.core.frame.DataFrame, dict]

,A.1 Total Costs,A.1_Expected Number of Deaths,A.1_Expected Annual Damage,A.2 Total Costs,A.2_Expected Number of Deaths,A.2_Expected Annual Damage,A.3 Total Costs,A.3_Expected Number of Deaths,A.3_Expected Annual Damage,A.4 Total Costs,A.4_Expected Number of Deaths,A.4_Expected Annual Damage,A.5 Total Costs,A.5_Expected Number of Deaths,A.5_Expected Annual Damage,RfR Total Costs,Expected Evacuation Costs
0,1.453776e+08,0.0,0.0,1.387250e+08,0.000000,0.000000e+00,1.005732e+08,0.000000,0.000000e+00,3.494362e+07,0.000000,0.000000e+00,9.675851e+07,0.0,0.0,1.305500e+09,0.000000
1,1.453776e+08,0.0,0.0,1.387250e+08,0.000000,0.000000e+00,1.367206e+08,0.009260,3.614739e+07,3.494362e+07,0.000000,0.000000e+00,9.675851e+07,0.0,0.0,1.305500e+09,2400.551979
2,1.453776e+08,0.0,0.0,1.387250e+08,0.000000,0.000000e+00,1.249213e+08,0.003923,2.434804e+07,3.494362e+07,0.000000,0.000000e+00,9.675851e+07,0.0,0.0,1.305500e+09,948.740778
3,1.453776e+08,0.0,0.0,1.517358e+08,0.001478,1.301082e+07,1.005732e+08,0.000000,0.000000e+00,3.494362e+07,0.000000,0.000000e+00,9.675851e+07,0.0,0.0,1.305500e+09,822.579376
4,1.453776e+08,0.0,0.0,1.387250e+08,0.000000,0.000000e+00,1.064878e+08,0.000969,5.914593e+06,3.494362e+07,0.000000,0.000000e+00,9.675851e+07,0.0,0.0,1.305500e+09,226.541969
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2.256261e+08,0.0,0.0,2.231666e+08,0.000851,2.451922e+06,7.012793e+07,0.000754,1.182343e+06,1.692524e+07,0.000000,0.000000e+00,1.420865e+08,0.0,0.0,1.051100e+09,86.337112
96,2.256261e+08,0.0,0.0,2.207147e+08,0.000000,0.000000e+00,1.035052e+08,0.026066,3.455966e+07,1.692524e+07,0.000000,0.000000e+00,1.420865e+08,0.0,0.0,1.051100e+09,967.880870
97,2.256261e+08,0.0,0.0,2.207147e+08,0.000000,0.000000e+00,8.400499e+07,0.011917,1.505940e+07,1.692524e+07,0.000000,0.000000e+00,1.420865e+08,0.0,0.0,1.051100e+09,394.727962
98,2.256261e+08,0.0,0.0,2.272626e+08,0.003396,6.547930e+06,8.114223e+07,0.011655,1.219664e+07,1.692524e+07,0.000000,0.000000e+00,1.420865e+08,0.0,0.0,1.051100e+09,649.286703


,0_RfR 0,0_RfR 1,0_RfR 2,1_RfR 0,1_RfR 1,1_RfR 2,2_RfR 0,2_RfR 1,2_RfR 2,3_RfR 0,...,A.3_DikeIncrease 0,A.3_DikeIncrease 1,A.3_DikeIncrease 2,A.4_DikeIncrease 0,A.4_DikeIncrease 1,A.4_DikeIncrease 2,A.5_DikeIncrease 0,A.5_DikeIncrease 1,A.5_DikeIncrease 2,EWS_DaysToThreat
0,1,0,0,1,0,1,0,1,0,1,...,3,4,8,7,1,4,3,2,3,4
1,1,0,0,1,0,1,0,1,0,1,...,3,4,8,7,1,4,3,2,3,4
2,1,0,0,1,0,1,0,1,0,1,...,3,4,8,7,1,4,3,2,3,4
3,1,0,0,1,0,1,0,1,0,1,...,3,4,8,7,1,4,3,2,3,4
4,1,0,0,1,0,1,0,1,0,1,...,3,4,8,7,1,4,3,2,3,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0,1,1,0,0,1,1,0,0,0,...,0,8,3,4,0,1,8,4,4,1
96,0,1,1,0,0,1,1,0,0,0,...,0,8,3,4,0,1,8,4,4,1
97,0,1,1,0,0,1,1,0,0,0,...,0,8,3,4,0,1,8,4,4,1
98,0,1,1,0,0,1,1,0,0,0,...,0,8,3,4,0,1,8,4,4,1


In [183]:
pd.concat([results[0].iloc[:,19:-3],  pd.DataFrame(results[1])], axis=1).columns

Index(['0_RfR 0', '0_RfR 1', '0_RfR 2', '1_RfR 0', '1_RfR 1', '1_RfR 2',
       '2_RfR 0', '2_RfR 1', '2_RfR 2', '3_RfR 0', '3_RfR 1', '3_RfR 2',
       '4_RfR 0', '4_RfR 1', '4_RfR 2', 'A.1_DikeIncrease 0',
       'A.1_DikeIncrease 1', 'A.1_DikeIncrease 2', 'A.2_DikeIncrease 0',
       'A.2_DikeIncrease 1', 'A.2_DikeIncrease 2', 'A.3_DikeIncrease 0',
       'A.3_DikeIncrease 1', 'A.3_DikeIncrease 2', 'A.4_DikeIncrease 0',
       'A.4_DikeIncrease 1', 'A.4_DikeIncrease 2', 'A.5_DikeIncrease 0',
       'A.5_DikeIncrease 1', 'A.5_DikeIncrease 2', 'EWS_DaysToThreat',
       'A.1 Total Costs', 'A.1_Expected Number of Deaths',
       'A.1_Expected Annual Damage', 'A.2 Total Costs',
       'A.2_Expected Number of Deaths', 'A.2_Expected Annual Damage',
       'A.3 Total Costs', 'A.3_Expected Number of Deaths',
       'A.3_Expected Annual Damage', 'A.4 Total Costs',
       'A.4_Expected Number of Deaths', 'A.4_Expected Annual Damage',
       'A.5 Total Costs', 'A.5_Expected Number of Deaths',


In [182]:
results1.columns

Index(['0_RfR 0', '0_RfR 1', '0_RfR 2', '1_RfR 0', '1_RfR 1', '1_RfR 2',
       '2_RfR 0', '2_RfR 1', '2_RfR 2', '3_RfR 0', '3_RfR 1', '3_RfR 2',
       '4_RfR 0', '4_RfR 1', '4_RfR 2', 'EWS_DaysToThreat',
       'A.1_DikeIncrease 0', 'A.1_DikeIncrease 1', 'A.1_DikeIncrease 2',
       'A.2_DikeIncrease 0', 'A.2_DikeIncrease 1', 'A.2_DikeIncrease 2',
       'A.3_DikeIncrease 0', 'A.3_DikeIncrease 1', 'A.3_DikeIncrease 2',
       'A.4_DikeIncrease 0', 'A.4_DikeIncrease 1', 'A.4_DikeIncrease 2',
       'A.5_DikeIncrease 0', 'A.5_DikeIncrease 1', 'A.5_DikeIncrease 2',
       'A.5_Expected Number of Deaths', 'A.5_Expected Annual Damage',
       'RfR Total Costs', 'Expected Evacuation Costs'],
      dtype='object')

In [ ]:
problem.nobjs

In [154]:
from ema_workbench.em_framework.optimization import to_problem

problem = to_problem(dike_model, searchover="levers")

reference_set = epsilon_nondominated(results, [0.05] * len(dike_model.outcomes), problem)

hv = HypervolumeMetric(reference_set, problem)

TypeError: cannot concatenate object of type '<class 'dict'>'; only Series and DataFrame objs are valid

In [ ]:
help(problem)

In [ ]:
help(reference_set)

In [ ]:
reference_set.problem

In [ ]:
from ema_workbench import MultiprocessingEvaluator, ema_logging
from ema_workbench.em_framework.evaluators import BaseEvaluator

ema_logging.log_to_stderr(ema_logging.INFO)
with SequentialEvaluator(dike_model) as evaluator:
    results1 = evaluator.optimize(nfe=1, searchover='levers',
                                  epsilons=[0.1, ] * len(dike_model.outcomes),
                                  reference=Scenarios[0])

In [ ]:
k = r"C:\Users\roelo\Videos\archive"


In [ ]:
all_archives = []

for i in range(2):
    archives = ArchiveLogger.load_archives(k+ "\\" + f"{i}.tar.gz")
    all_archives.append(archives)

In [ ]:
from optimization_tuned import epsilon_nondominated,HypervolumeMetric

In [ ]:
from optimization_tuned import (
    GenerationalDistanceMetric,
    EpsilonIndicatorMetric,
    InvertedGenerationalDistanceMetric,
    SpacingMetric,
)
from ema_workbench.em_framework.optimization import to_problem

problem = to_problem(dike_model, searchover="levers")

reference_set = epsilon_nondominated(results, [0.05] * len(dike_model.outcomes), problem)

In [ ]:
from optimization_tuned import (
    HypervolumeMetric,
    GenerationalDistanceMetric,
    EpsilonIndicatorMetric,
    InvertedGenerationalDistanceMetric,
    SpacingMetric,
)
from ema_workbench.em_framework.optimization import to_problem

problem = to_problem(dike_model, searchover="levers")

reference_set = epsilon_nondominated(results, [0.05] * len(dike_model.outcomes), problem)

hv = HypervolumeMetric(reference_set, problem)
gd = GenerationalDistanceMetric(reference_set, problem, d=1)
ei = EpsilonIndicatorMetric(reference_set, problem)
ig = InvertedGenerationalDistanceMetric(reference_set, problem, d=1)
sm = SpacingMetric(problem)


metrics_by_seed = []
for archives in all_archives:
    metrics = []
    for nfe, archive in archives.items():
        scores = {
            "generational_distance": gd.calculate(archive),
            "hypervolume": hv.calculate(archive),
            "epsilon_indicator": ei.calculate(archive),
            "inverted_gd": ig.calculate(archive),
            "spacing": sm.calculate(archive),
            "nfe": int(nfe),
        }
        metrics.append(scores)
    metrics = pd.DataFrame.from_dict(metrics)

    # sort metrics by number of function evaluations
    metrics.sort_values(by="nfe", inplace=True)
    metrics_by_seed.append(metrics)

### OOOOOOOOOOOOOOOOOOOOOOOOOOOO


In [ ]:
from ema_workbench import MultiprocessingEvaluator, ema_logging
from ema_workbench.em_framework.evaluators import BaseEvaluator

ema_logging.log_to_stderr(ema_logging.INFO)

with SequentialEvaluator(dike_model) as evaluator:
    results1 = evaluator.optimize(nfe=1000, searchover='levers',
                                 epsilons=[0.1,]*len(dike_model.outcomes),reference=Scenarios[0])

In [ ]:
list(dike_model.outcomes)

In [ ]:
df = pd.DataFrame(results1)

In [ ]:
df["Expected Number of Deaths"] = df["A.1_Expected Number of Deaths"] + df["A.2_Expected Number of Deaths"] + df[
    "A.3_Expected Number of Deaths"] + df["A.4_Expected Number of Deaths"] + df["A.5_Expected Number of Deaths"]
df["Expected Annual Damage"] = df["A.1_Expected Annual Damage"] + df["A.2_Expected Annual Damage"] + df[
    "A.3_Expected Annual Damage"] + df["A.4_Expected Annual Damage"] + df["A.5_Expected Annual Damage"]


## Feature importance (not required but nice to know)
df_feature_importance_out = df[["Expected Evacuation Costs", "Expected Annual Damage", "Expected Number of Deaths","A.5_Expected Number of Deaths", "A.5_Expected Annual Damage"]]

In [ ]:
from ema_workbench.analysis import parcoords

# data = results1.loc[:, [o.name for o in dike_model.outcomes]]
limits = parcoords.get_limits(df_feature_importance_out)
limits.loc[0, ["Expected Evacuation Costs","A.5_Expected Number of Deaths","A.5_Expected Annual Damage"]] = 0

paraxes = parcoords.ParallelAxes(limits)
paraxes.plot(df_feature_importance_out)
# paraxes.invert_axis('max_P')
plt.show()

In [ ]:
results1

In [ ]:
data